# Interactive Assessment Graphs
To make the findings of our module assessment information breakdown easier, we have proposed an interactive graphing tool for ease of use in finding the difference between schools, level and modules. This Jupyter Notebook is for testing and development of said tool.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pathlib as Path
import plotly.graph_objects as go
import ipywidgets 
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
from IPython.display import display, clear_output
from plotly.subplots import make_subplots
import matplotlib.ticker as mtick

In [2]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter serverextension enable voila --sys-prefix

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling: voila
- Writing config: C:\Users\catha\anaconda3\etc\jupyter
    - Validating...
      voila 0.4.0 ok


In [3]:
#This finds files that were saved by the Module Scraper Code
def file_finder(school=None, level=None, givenList=None, filename=None, module=None):
    #Set the path to the overall directory with our input files
    dir_raw=Path.Path("ModuleInformation")
    
    #This locates the subdirectory where files by school, level or list can be found
    subdirectory=""
    #Set the subdirectory 
    if level != None:
        subdirectory+= "Level=%d" %(level)
    if school != None:
        subdirectory+= "_School="+school.replace(" ", "-")
    if filename != None:
        subdirectory = filename
    if module != None:
        subdirectory = ("IndividualModules/%s" %module)
        
    #If a subdirectory has indeed been set, change the path to reflect this
    if len(subdirectory) > 0:
        dir_raw =dir_raw / subdirectory
        

    #Read the files for the assessment and descriptors in, closing the file immediately after
    with open(dir_raw / "assessments.json", 'r') as infile:
        #print("Reading from %s" % dir_raw)
        if module != None:
            assessments=pd.read_json(infile, orient="columns")
        else:
            assessments=pd.read_json(infile)
    with open(dir_raw / "descriptors.json", 'r') as infile:
        #print("Reading from %s" % dir_raw)
        if module != None:
            descriptors=pd.read_json(infile, typ="series")
        else:
            descriptors=pd.read_json(infile)

    assessments["Level"]=assessments["Level"].apply(lambda x: pd.to_numeric(x.split('(')[0], errors='ignore'))
    #This is the dataset cleaning, so that they are ready for presentation
    assessments["% of Final Grade"]=pd.to_numeric(assessments["% of Final Grade"], errors='coerce')
    assessments=assessments.replace("Multiple Choice Questionnaire (Short)", "Multiple Choice Questionnaire")
    assessments["Assessment Type"]=assessments["Assessment Type"].astype("category")
    assessments["Student Credits"]=((assessments["Scaled % of Final Grade"]/100)*5)*assessments["Enrolled Students 22/23"]
    assessments["Assessment Credits"]=((assessments["Scaled % of Final Grade"]/100)*5)
    #Return the desired assessment and description information
    return assessments, descriptors

In [4]:
susceptibility={"Assignment" :"susceptible", \
                "Attendance": "not susceptible", \
                "Class Test" : "not susceptible", \
                "Continuous Assessment": "susceptible", \
               "Essay": "susceptible", \
                "Examination": "not susceptible", \
                "Fieldwork": "not susceptible", \
                "Group Project": "susceptible", \
                "Journal": "not susceptible",\
               "Lab Report": "not susceptible", \
                "Multiple Choice Questionnaire": "not susceptible", \
                "Oral Examination": "not susceptible", \
               "Portfolio" : "susceptible",  \
                "Practical Examination": "not susceptible", \
                "Presentation" : "not susceptible", \
                "Project": "susceptible", \
               "Seminar": "not susceptible", \
               "Studio Examination" : "not susceptible",\
               "Assessments worth <2%": "not susceptible"}

In [5]:
work_type={"Assignment" :"At home", \
                "Attendance": "In person", \
                "Class Test" : "In person", \
                "Continuous Assessment": "At home", \
               "Essay": "At home", \
                "Examination": "In person", \
                "Fieldwork": "In person", \
                "Group Project": "Blended", \
                "Journal": "Blended",\
               "Lab Report": "Blended", \
                "Multiple Choice Questionnaire": "Blended", \
                "Oral Examination": "In person", \
               "Portfolio" : "Blended",  \
                "Practical Examination": "In person", \
                "Presentation" : "In person", \
                "Project": "At home", \
               "Seminar": "In person", \
               "Studio Examination" : "In person",\
               "Assessments worth <2%": "Unknown"}

In [6]:
#Test the file finder function 
assessments, descriptors = file_finder()

#Builds a graph to indicate the susceptibility of a module set's assessments to Chat GPT
def workType_builder(Weighted=True, School="All", Level=0, module="", Path=None, time=False,\
                     student_credits=False, with_internships=True):
    ################################################################################
    work_type={"Assignment" :"At home", \
                "Attendance": "In person", \
                "Class Test" : "In person", \
                "Continuous Assessment": "At home", \
               "Essay": "At home", \
                "Examination": "In person", \
                "Fieldwork": "In person", \
                "Group Project": "Blended", \
                "Journal": "Blended",\
               "Lab Report": "Blended", \
                "Multiple Choice Questionnaire": "Blended", \
                "Oral Examination": "In person", \
               "Portfolio" : "Blended",  \
                "Practical Examination": "In person", \
                "Presentation" : "In person", \
                "Project": "At home", \
               "Seminar": "In person", \
               "Studio Examination" : "In person",\
               "Assessments worth <2%": "Unknown"}
    #################################################################################
    
    new_assessments=assessments
    

    if Path != None:
        path=Path.replace(" ", "")
        new_assessments, descriptors = file_finder(filename=path)
    if with_internships != True:
        internship_file= "Datasets/UCD_EngArch_InternshipModules.xlsx"
        internships=pd.read_excel(internship_file)

        new_assessments=new_assessments[~new_assessments["Module Code"].isin(internships["Code"])]
        
    #First we will need to filter the dataset as desired
    if School != "All":
        new_assessments=new_assessments[new_assessments["School"]==School]
    if Level != 0:
        new_assessments=new_assessments[new_assessments["Level"]==Level]
    if len(module) > 0:
        new_assessments=new_assessments[new_assessments["Module Code"]==module]
        
    if len(new_assessments)==0:
        print("***********ERROR: No modules found with given parameters*************")
        
    num_unique=len(new_assessments["Module Code"].unique())
    
    #Data Cleaning and Manipulation Part
    #########################################################################################
    #First we will reduce the assessments dataframe to just the columns we require here.

    #For the pie chart, group by susceptibilty
    assessment_sus=new_assessments.groupby(by="Work Type").sum()
    #For the bar chart, group by assessment type
    assessment_total=new_assessments.groupby(by="Assessment Type").sum()

    #Choose whether or not you want to scale by credits, or take all modules to be equal, regardless of credits
    if student_credits==True:
        assessment_scale="Student Credits"
    elif Weighted==True:
        assessment_scale="Scaled % of Final Grade"
    else:
        assessment_scale="% of Final Grade"
        
        
    total=assessment_sus[assessment_scale].sum()

    #Get the percentage each assessment type makes up of the overall for both graphs
    assessment_sus["% of Assessment"]=assessment_sus[assessment_scale].apply(lambda x: x/total)
    assessment_total["% of Assessment"]=assessment_total[assessment_scale].apply(lambda x: x/total)

    #The groupby function removed the categorical "Susceptibility" column. Bring it back
    assessment_total["Work Type"]=assessment_total.index.map(lambda x: work_type[x])

    #Sort the totals into descending order, to make it easier to see
    sorted_totals=assessment_total.sort_values(by="% of Assessment", ascending=False)
    #The groupby function removed the categorical "Susceptibility" column. Bring it back
    sorted_totals["Work Type"]=sorted_totals.index.map(lambda x: work_type[x])

    #Graph building part
    #############################################################################################################
    #Set the font
    plt.rcParams["font.family"] = "sans"
    
    #Make the figure
    fig, (ax, bx)=plt.subplots(ncols=2, figsize=(16, 9))

    #Make a pie chart on the desired axes
    if time == False:
        fig, ax=workType_pie(assessment_sus, fig, ax, num_unique)
    else:
        stage=False
        if Path != None:
            stage=True
        #Make the area level stacked chart
        fig, ax=workType_stackedBar(new_assessments, fig, ax, stage=stage, student_credits=student_credits)
        
    #Make the horizontal bar chart on the desired axes
    fig, bx=workType_barh(sorted_totals, fig, bx)
    
    #Here we want to set the overall title of the charts - to indicate what module subset they are describing
    what="College of Engineering and Architecture"
    if School != 'All':
        what=""
        what+="School of "
        what+=School
    if Level != 0:
        what+= " At Level %d" %Level
    if Path != None:
        what="".join([(" "+i if i.isupper() else i) for i in Path])
    if len(module) > 0:
        what = module + " Module"
    if not with_internships:
        what = what + " Excluding Internships"
    if not time: 
        what = what + " Snapshot"
        
    #Set the overall title for the two graphs
    fig.suptitle("Assessment Breakdown in the %s" %what, fontsize=24)
    fig.tight_layout()


FileNotFoundError: [Errno 2] No such file or directory: 'ModuleInformation\\assessments.json'

In [ ]:
def workType_pie(assessment_sus, fig, ax, num_unique=None):
        #These are for selecting colours and hatching for each pie segment
    colour_dict={"At home": "#1f78b4", "Blended": "#a6cee3", "In person":"#b2df8a"}
    hatching_dict={"At home": None, "Blended": None, "In person":None}
    colours=[]
    hatching=[]
    #Get the colours and hatching for each pie segment
    for segment in assessment_sus.index:
        colours.append(colour_dict[segment])
        hatching.append(hatching_dict[segment])

    #Make the susceptibility pie chart
    ax.pie(assessment_sus["% of Assessment"],\
           wedgeprops = { 'linewidth' : 1.5, 'edgecolor' : 'white', "alpha" : 0.9}, \
           autopct="%1.1f%%", labels=assessment_sus.index, hatch=hatching ,\
           colors=colours, textprops={'fontsize':18})

    #Set the title of the pie chart for ease of understanding
    ax.set_title("Proportions of Different Assessment Kinds", fontsize=20)
    
    if num_unique!=None:
        ax.set_xlabel("\n\nNumber of Modules in this Breakdown: %d" %(num_unique), fontsize=20)
    
    return fig, ax

In [ ]:
def workType_barh(sorted_totals, fig, bx):
    #These are for selecting colours and hatching for each bar in the bar chart
    colour_dict={"At home": "#1f78b4", "Blended": "#a6cee3", "In person":"#b2df8a"}
    hatching_dict={"At home": None, "Blended": None, "In person":None}
    colours=[]
    hatching=[]
    #Get the colours and hatching type based on susceptibilty
    for sus in sorted_totals["Work Type"]:
        colours.append(colour_dict[sus])
        hatching.append(hatching_dict[sus])
        
        
    #Make the assessment type breakdown bar chart, with colour and hatching indicating the susceptibility
    bx.barh(sorted_totals.index, sorted_totals["% of Assessment"].apply(lambda x:x*100), edgecolor="white", hatch = hatching ,\
           color=colours, alpha=0.9)
    
    #colors = {'fruit':'red', 'veggie':'green'}         
    #Set the options for the axes
    bx.tick_params(axis='y', labelsize=16)
    bx.tick_params(axis='x', labelsize=14)
    
    fmt = '%d%%' # Format you want the ticks, e.g. '40%'
    xticks = mtick.FormatStrFormatter(fmt)
    bx.xaxis.set_major_formatter(xticks)
    #ax.legend(fontsize=20)
    bx.set_xlabel("Percentage of Assessment",fontsize=18)
    #Set the title for the bar chart for ease of understanding
    bx.set_title("Assessment Type Breakdown", fontsize=20)
    
    #Do the legend
    labels = list(colour_dict.keys())
    handles = [plt.Rectangle((0,0),1,1, color=colour_dict[label]) for label in labels]
    bx.legend(handles, labels,fontsize=20)
    
    return fig, bx

In [ ]:
def workType_stackedBar(assessments, fig, ax, stage=False, student_credits=False):
    #Choose whether or not you want to scale by credits, or take all modules to be equal, regardless of credits
    if student_credits==True:
        assessment_scale="Student Credits"
    else:
        assessment_scale="Assessment Credits"
        
    if stage==False:
        time="Level"
    else:
        time="Stage"
        
    #assessments[time]=assessments[time].apply(lambda x: "%d" %x)
    categories=np.array(assessments[time].unique())
    
    assessments[time]=pd.Categorical(assessments[time], categories=categories, ordered=True)
    stackedbar=assessments.groupby(by=["Work Type", time], as_index=False).sum()

    #These are for selecting colours and hatching for each area segment
    colour_dict={"At home": "#1f78b4", "Blended": "#a6cee3", "In person":"#b2df8a"}
    hatching_dict={"At home": None, "Blended": None, "In person":None}
    colours=[]
    Stackedwork={}

    
    #Get the colours and hatching for each area segment
    for i, segment in enumerate(stackedbar["Work Type"].unique().tolist()):
        Stackedwork[segment]=stackedbar[stackedbar["Work Type"] == segment][assessment_scale].reset_index(drop=True)
        if i ==0:
            Totalwork=Stackedwork[segment]
        else:
            Totalwork=Totalwork+Stackedwork[segment]
                                         
    #If any work type are not included in the stacked bar, add in a value for that segment of 0
    for work in ["At home", "Blended", "In person"]: 
        if work not in Stackedwork.keys():
                Stackedwork[work]=np.zeros(len(assessments[time].unique().tolist()))

            
 
    ax.bar(["{}".format(level) for level in assessments[time].unique().tolist()],\
        (Stackedwork["At home"]), color=colour_dict["At home"])
        
    ax.bar(["{}".format(level) for level in assessments[time].unique().tolist()],\
           (Stackedwork["Blended"]), \
           bottom=(Stackedwork["At home"]), color=colour_dict["Blended"])
    
    ax.bar(["{}".format(level) for level in assessments[time].unique().tolist()],\
           (Stackedwork["In person"]), \
           bottom=(Stackedwork["Blended"])+(Stackedwork["At home"]), color=colour_dict["In person"])

    # Add percentages as labels to the stacked bars
    for idx in assessments[time].unique().tolist():
        start = 0
        for work in ["At home", "Blended", "In person"]:
            y=Stackedwork[work][idx-1]
            value = Stackedwork[work][idx-1]
            total = Totalwork[idx-1]
            if  y/max(Totalwork)> .05 :
                ax.text(
                x=idx-1,
                y=start + y / 2,
                s=f"{round(100 * value / total, 1)}%",
                fontsize=16,
                ha="center"
                )
            start += y
            
    #Set the options for the axes
    ax.tick_params(axis='y', labelsize=16)
    ax.tick_params(axis='x', labelsize=16)
    
    #ax.legend(fontsize=20)
    ax.set_ylabel("Taught Student Credits",fontsize=18)
    ax.set_xlabel("%s" %time,fontsize=18)
    #ax.legend(["At home", "Hybrid", "In person"],fontsize=20)
    #Set the title for the bar chart for ease of understanding
    ax.set_title("Assessment Proportions over %s" %time , fontsize=20)
    
    return fig, ax

In [ ]:
def treemap(assessments, school=None, taught_credits=True):
    import plotly.io as pio
    
    new_assessments=assessments
    title=""
    #If a school is specified, make the treemap just for that school
    if school!=None:
        new_assessments=assessments[assessments["School"]==school]
        title="of " + school
        
        
        
    color_dict={"Civil Engineering":"#AB63FA", \
                     "Mechanical & Materials Eng": "#EF553B", \
                     "Chem & Bioprocess Engineering": "#FFA15A", \
                     "Biosystems & Food Engineering": "#19D3F3", \
                     "Architecture, Plan & Env Pol": "#636EFA", \
                     "Electrical & Electronic Eng": "#00CC96"}
    
    #Group the modules as we want to
    new_assessments["Module Level"]=assessments["Level"].apply(lambda x: "Level %d Modules" %x)
    modules=new_assessments.groupby(by=["School", "Module Level", "Module Code"], as_index=False).sum()
    if taught_credits:
        credits="Student Credits"
    else:
        credits="Credits"
    modules=modules[modules[credits] != 0]
    
    #Set the figure up
    fig = px.treemap(modules, path=[\
                               px.Constant("College of Engineering and Architecture - Modules offered by School %s" %(title)), \
                               "School", 'Module Level', 'Module Code'], values=credits, \
                     color_discrete_map=[color_dict[school] for school in modules["School"].unique()])
    
    fig.update_layout( margin=dict(t=25, l=25, r=25, b=25))#, annotations=list(x = 1, y = -0.1,\
                                                                             #text = "Source: data I found somewhere.",
                                                                             #showarrow = F,\
                                                                             #xref='paper', yref='paper', \
                                                                             #xanchor='right',\
                                                                             #yanchor='auto', \
                                                                             #xshift=0, \
                                                                             #yshift=0)#,
                                                                             #font=list(size=15, color="red"))
                                                                             #)
    fig.show()
    
    #Set the path to save the graphs
    dir_output=Path.Path("Graphs")
    
    #Set the filename for th figure
    dir_output=dir_output / "ModuleTreemap"
    dir_output.mkdir(parents=True, exist_ok=True)
    
    filename= dir_output / "College.png"
    pio.write_image(fig, filename, width=1600, height=900, scale=3)
    
    filename= dir_output / "College.pdf"
    pio.write_image(fig, filename, width=1600, height=900, scale=3)
    
treemap(assessments)

In [ ]:
school_list=assessments["School"].unique().tolist()
school_list.append("All")

module_list=assessments["Module Code"].unique().tolist()
module_list.append("")

dir_raw= Path.Path("ModuleInformation")

path_list=[]
for file in dir_raw.glob("*"):
    if "Path" in str(file):
        filename="".join([(" "+i if i.isupper() else i) for i in str(file)])
        filename= filename.split('\\')[1]
        path_list.append(filename)

path_list.append(None)


moduleGetter=ipywidgets.Combobox(
    # value='John',
    placeholder='Enter Module Code',
    options=module_list,
    description='Module:',
    ensure_option=True,
)

ipywidgets.interact_manual(workType_builder, School=school_list, Level=(0, 5, 1), module=moduleGetter, Path=path_list)

In [ ]:
#Builds a graph to indicate the susceptibility of a module set's assessments to Chat GPT
def workType_plotly(Weighted=True, School="All", Level=0, module="", Path=None, time=False):
    ################################################################################
    work_type={"Assignment" :"At home", \
                "Attendance": "In person", \
                "Class Test" : "In person", \
                "Continuous Assessment": "At home", \
               "Essay": "At home", \
                "Examination": "In person", \
                "Fieldwork": "In person", \
                "Group Project": "Blended", \
                "Journal": "Blended",\
               "Lab Report": "Blended", \
                "Multiple Choice Questionnaire": "Blended", \
                "Oral Examination": "In person", \
               "Portfolio" : "Blended",  \
                "Practical Examination": "In person", \
                "Presentation" : "In person", \
                "Project": "At home", \
               "Seminar": "In person", \
               "Studio Examination" : "In person",\
               "Assessments worth <2%": "Unknown"}
    #################################################################################
    
    new_assessments=assessments
    if Path != None:
        path=Path.replace(" ", "")
        new_assessments, descriptors = file_finder(filename=path)
        
    #First we will need to filter the dataset as desired
    if School != "All":
        new_assessments=new_assessments[new_assessments["School"]==School]
    if Level != 0:
        new_assessments=new_assessments[new_assessments["Level"]==Level]
    if len(module) > 0:
        new_assessments=new_assessments[new_assessments["Module Code"]==module]
        
    if len(new_assessments)==0:
        print("***********ERROR: No modules found with given parameters*************")
        
    num_unique=len(new_assessments["Module Code"].unique())
    
    #Data Cleaning and Manipulation Part
    #########################################################################################

    #For the pie chart, group by susceptibilty
    assessment_sus=new_assessments.groupby(by="Work Type").sum()
    #For the bar chart, group by assessment type
    assessment_total=new_assessments.groupby(by="Assessment Type").sum()

    #Choose whether or not you want to scale by credits, or take all modules to be equal, regardless of credits
    if Weighted==True:
        assessment_scale="Scaled % of Final Grade"
    else:
        assessment_scale="% of Final Grade"
        
    total=assessment_sus[assessment_scale].sum()

    #Get the percentage each assessment type makes up of the overall for both graphs
    assessment_sus["% of Assessment"]=assessment_sus[assessment_scale].apply(lambda x: x/total)
    assessment_total["% of Assessment"]=assessment_total[assessment_scale].apply(lambda x: x/total)

    #The groupby function removed the categorical "Susceptibility" column. Bring it back
    assessment_total["Work Type"]=assessment_total.index.map(lambda x: work_type[x])

    #Sort the totals into descending order, to make it easier to see
    sorted_totals=assessment_total.sort_values(by="% of Assessment", ascending=False)
    #The groupby function removed the categorical "Susceptibility" column. Bring it back
    sorted_totals["Work Type"]=sorted_totals.index.map(lambda x: work_type[x])

    #Graph building part
    #############################################################################################################
    #Make the figure with subplots
    if time == False:
        fig= make_subplots(
        cols=2,
        column_widths=[0.5, 0.5],
        specs=[[{"type": "pie"}, {"type": "bar"}]])
        
        #Add the plotly pie trace
        fig=plotly_pie(fig, assessment_sus)

        #Add the plotly barh trace
        fig=plotly_barh(fig, sorted_totals)
    else:
        fig= make_subplots(
        cols=2,
        column_widths=[0.5, 0.5],
        specs=[[{"type": "bar"}, {"type": "bar"}]])
        
        #Add the plotly pie trace
        fig=plotly_stackedbar(fig, new_assessments)

        #Add the plotly barh trace
        fig=plotly_barh(fig, sorted_totals)
        
    

    #Here we want to set the overall title of the charts - to indicate what module subset they are describing
    what="College of Engineering and Architecture"
    if Path != None:
        what= Path
    if School != 'All':
        what=""
        what+="School of "
        what+=School
    if Level != 0:
        what+= " At Level %d" %Level
    if len(module) > 0:
        what = module + " Module"
        
    #Set the overall title for the two graphs
    #fig.suptitle("Assessment Breakdown in the %s" %what, fontsize=24)
    #fig.tight_layout()
           
    fig.show()

In [ ]:
def plotly_pie(fig, assessment_sus):
    #These are for selecting colours and hatching for each pie segment
    colour_dict={"At home": "#1f78b4", "Blended": "#a6cee3", "In person":"#b2df8a"}

    colours=[]
    
    #Get the colours and hatching for each pie segment
    for segment in assessment_sus.index:
        colours.append(colour_dict[segment])

    #Build the plotly pie chart trace
    fig.add_trace(
    go.Pie(values=assessment_sus["% of Assessment"],
            labels=assessment_sus.index,
           marker=dict(colors=colours),
           #pattern=dict(shape=hatching),
            showlegend=True),
    row=1, col=1
    )
    
    return fig

In [ ]:
def plotly_barh(fig, sorted_totals):
    #These are for selecting colours and hatching for each bar in the bar chart
    colour_dict={"At home": "#1f78b4", "Blended": "#a6cee3", "In person":"#b2df8a"}
    colours=[]

    #Get the colours and hatching type based on susceptibilty
    for sus in sorted_totals["Work Type"]:
        colours.append(colour_dict[sus])

        
    fig.add_trace(
    go.Bar(x=sorted_totals.index, y=sorted_totals["% of Assessment"].apply(lambda x: x*100),\
            marker=dict(color=colours),
           #color=sorted_totals["Work Type"],
           #legendgroup="Work Type",
            showlegend= False, orientation='v'),
    row=1, col=2
    )
    
    return fig

In [ ]:
def plotly_stackedbar(fig, assessments, stage=False, student_credits=False):

    #Choose whether or not you want to scale by credits, or take all modules to be equal, regardless of credits
    if student_credits==True:
        assessment_scale="Student Credits"
    else:
        assessment_scale="Assessment Credits"
        
    if stage==False:
        time="Level"
    else:
        time="Stage"
        
    categories=np.array(assessments[time].unique())
    
    assessments[time]=pd.Categorical(assessments[time], categories=categories, ordered=True)
    stackedbar=assessments.groupby(by=["Work Type", time], as_index=False).sum()

    #These are for selecting colours and hatching for each area segment
    colour_dict={"At home": "#1f78b4", "Blended": "#a6cee3", "In person":"#b2df8a"}
    #hatching_dict={"At home": None, "Blended": None, "In person":None}
    colours=[]
    Stackedwork={}

    
    #Get the colours and hatching for each area segment
    for i, segment in enumerate(stackedbar["Work Type"].unique().tolist()):
        Stackedwork[segment]=stackedbar[stackedbar["Work Type"] == segment][assessment_scale].reset_index(drop=True)
        if i ==0:
            Totalwork=Stackedwork[segment]
        else:
            Totalwork=Totalwork+Stackedwork[segment]
                                         
    #If any work type are not included in the stacked bar, add in a value for that segment of 0
    for work in ["At home", "Blended", "In person"]: 
        if work not in Stackedwork.keys():
                Stackedwork[work]=np.zeros(len(assessments[time].unique().tolist()))

    fig.add_trace(
    go.Bar(x=["{}".format(level) for level in assessments[time].unique().tolist()],\
            y=Stackedwork["At home"], marker_color="#1f78b4"),
    row=1, col=1
    )
    fig.add_trace(        
    go.Bar(x=["{}".format(level) for level in assessments[time].unique().tolist()],\
               y=Stackedwork["Blended"], \
               base=(Stackedwork["At home"]), marker_color="#a6cee3"),
        row=1, col=1)
    fig.add_trace(
    go.Bar(x=["{}".format(level) for level in assessments[time].unique().tolist()],\
               y=Stackedwork["In person"], \
               base=(Stackedwork["Blended"])+(Stackedwork["At home"]), marker_color="#b2df8a"),
        row=1, col=1
    )
    
    return fig

In [ ]:
#workType_plotly(time=True)